# Credit scoring

**The task of the project is to define a portrait of a creditworthy client. Credit scoring is used by lenders to decide whether to extend or deny a loan.**

## Results

 - Customers who who take out a loan for transactions with residential or commercial real estate are noticeably more likely to repay loans on time than those who take out a loan for education or a car. The group of customers who take out a loan for a wedding is noticeably smaller than the other three groups, it is not correct to take this group into account.
 - Customers with children have a worse credit history than customers without children. 
 - Customers who are currently or in the past were officially married are more likely to repay loans. 
 - Men are less likely to repay their loans on time than women. Among the borrowers the number of women is 2 times more than men. 
 - People with the degree are more likely to repay the loan on time than people without.

**A business can use this research to develop personalized lending rates or repayment terms, according to the identified correlations.**

## Data understanding

In [1]:
#import libraries
import pandas as pd
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem
from collections import Counter

In [2]:
#open a table and its concise summary
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
#generate descriptive statistics
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [4]:
#open the first 15 rows
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


## Preprocessing

In [5]:
#find the rows with NaNs
print(data[data['days_employed'].isna() & ~ data['total_income'].isna()].empty)

True


In [6]:
#use fillna() to fill in the gaps and do a check
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))
print(data['total_income'].isna().sum())

0


In [7]:
#take a module from negative values and do a check
data['days_employed'] = data['days_employed'].abs()
print(data[data['days_employed'] < 0].empty)

True


In [8]:
#use fillna() to fill in the gaps and do a check
data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('mean'))
print(data.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


We assume that the age of the customer is equal to or greater than 18. We replace missing values with the average value of the cluster, since the age feature consists of homogeneous data.

In [9]:
#write te function to replace nulls, apply it to the rows and do a check
data_filtered = data[data['dob_years'] != 0].groupby('income_type')['dob_years'].mean()
 
def mean_age(row):
    if row['dob_years'] == 0:
        return data_filtered.loc[row['income_type']]
    return row['dob_years']
 
data['dob_years'] = data.apply(mean_age, axis=1)

print(data[data['dob_years'] < 18].empty)

True


We assume that the value of the days_employed feature cannot exceed 20000 - (75-20)*365, where 75 is the maximum value of the age feature.

In [10]:
#analyze where garbage is
display(data[data['days_employed'] > 20000].groupby('income_type')['income_type'].count())

income_type
безработный       2
пенсионер      3856
Name: income_type, dtype: int64

In [11]:
#analyze the unemployed
display(data[data['income_type'] == 'безработный'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31.0,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,395302.838654,45.0,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


In [12]:
#replace the incorrect values by the means and do a check
days_employed_20s_mean = data[(data['income_type'] != 'пенсионер') & (data['dob_years'] >= 18) & (data['dob_years'] < 30)]['days_employed'].mean()
days_employed_30s_mean = data[(data['income_type'] != 'пенсионер') & (data['income_type'] != 'безработный') & (data['dob_years'] >= 30) & (data['dob_years'] < 40)]['days_employed'].mean()
days_employed_40s_mean = data[(data['income_type'] != 'пенсионер') & (data['income_type'] != 'безработный') & (data['dob_years'] >= 40) & (data['dob_years'] < 50)]['days_employed'].mean()
days_employed_50s_mean = data[(data['income_type'] != 'пенсионер') & (data['dob_years'] >= 50) & (data['dob_years'] < 60)]['days_employed'].mean()
days_employed_60_75_mean = data[(data['income_type'] != 'пенсионер') & (data['dob_years'] >= 60) & (data['dob_years'] < 76)]['days_employed'].mean()

def work_experience(row):
    work_type = row['income_type']
    age = row['dob_years']
    if work_type == 'безработный':
        if age < 40:
            return days_employed_30s_mean
        return days_employed_40s_mean
    if work_type == 'пенсионер':
        if age >= 18 and age < 30:
            return days_employed_20s_mean
        if age >= 30 and age < 40:
            return days_employed_30s_mean
        if age >= 40 and age < 50:
            return days_employed_40s_mean
        if age >= 50 and age < 60:
            return days_employed_50s_mean
        return days_employed_60_75_mean
    return row['days_employed']

data['days_employed'] = data.apply(work_experience, axis=1)

display(data.head())
display(data[data['income_type'] == 'безработный'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,3165.910371,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,2058.104489,31.0,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,2691.130356,45.0,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


In [13]:
#change incorrect values and do a check
data.loc[data['children'] < 0, 'children'] = 0
data.loc[data['children'] == 20, 'children'] = 2
print(data[data['children'] > 5].empty)

True


In [14]:
#analyze the unique values and return the index of the row with the missing value
print(data['gender'].value_counts())
print(data[data['gender'] == 'XNA'].index)

F      14236
M       7288
XNA        1
Name: gender, dtype: int64
Int64Index([10701], dtype='int64')


In [15]:
#delete the row with the incorrect value and do a check
data = data.drop([10701])
print(data['gender'].value_counts()) 

F    14236
M     7288
Name: gender, dtype: int64


In [16]:
#change the register and do a check
data['education'] = data['education'].str.lower()
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,3165.910371,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [17]:
#return the unique values
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4176
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [18]:
#change the incorrect value and do a check
data.loc[data['family_status'] == 'Не женат / не замужем', 'family_status'] = 'не женат / не замужем'
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4176
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [19]:
#change data types and do a check
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')
data['dob_years'] = data['dob_years'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null int64
purpose             21524 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


In [20]:
#look for duplicates
data.duplicated().sum()

71

In [21]:
#delete the duplicates and do a check
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

### Lemmatization

In [22]:
#return the unique values
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка своего жилья                      620
покупка недвижимости                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [23]:
#return lemmas
text = ' '.join(data['purpose'])
lemmas = Mystem().lemmatize(text)
print(Counter(lemmas))

Counter({' ': 55021, 'недвижимость': 6350, 'покупка': 5896, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


In [24]:
#rewrite values of the column
m = Mystem()
def purposes(purpose):
    purpose_lemmatized = m.lemmatize(purpose)
    if 'автомобиль' in purpose_lemmatized:
        return 'автомобиль'
    if 'образование' in purpose_lemmatized:
        return 'образование'
    if 'свадьба' in purpose_lemmatized:
        return 'свадьба'
    if ('недвижимость' in purpose_lemmatized) or ('жилье' in purpose_lemmatized):
        return 'недвижимость'
    return 'другое'
data['purpose'] = data['purpose'].apply(purposes)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,3165,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


### Data clustering

In [25]:
#rewrite the values
def kids(row):
    value = row['children']
    if value > 0:
        return 'yes'
    return 'no'
data['children'] = data.apply(kids, axis=1)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,yes,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,yes,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,no,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,yes,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,no,3165,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


In [26]:
#rename the values
def income(row):
    value = row['total_income']
    if 20000 < value < 70000:
        return 'lowest'
    if 70000 <= value < 100000:
        return 'lower middle'
    if 100000 <= value < 200000:
        return 'middle'
    if 200000 <= value < 300000:
        return 'upper middle'
    return 'highest'

data['total_income'] = data.apply(income, axis=1)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,yes,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,upper middle,недвижимость
1,yes,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,middle,автомобиль
2,no,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,middle,недвижимость
3,yes,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,upper middle,образование
4,no,3165,53,среднее,1,гражданский брак,1,F,пенсионер,0,middle,свадьба


## Finding correlations

#### Is there a relationship between having children and repaying a loan on time?

In [27]:
#create a pivot table and a new column in it
data_kids = data.groupby('children').agg({'debt': ['sum','count']})
data_kids['ratio'] = (data_kids['debt']['sum']/data_kids['debt']['count'])*100
display(data_kids.style.format({('ratio', ''):'{:.2f}%'}))

Customers with children have a worse credit history than customers without children.

#### Is there a relationship between marital status and loan repayment on time?

In [28]:
#create a pivot table and a new column in it
data_family = data.pivot_table(index='family_status', values='debt', aggfunc=['sum','count'])
data_family['ratio'] = (data_family[('sum','debt')] / data_family[('count','debt')]) * 100
display(data_family.sort_values(by='ratio').style.format({('ratio', ''):'{:.2f}%'}))

,sum,count,ratio
,debt,debt,
family_status,,,
вдовец / вдова,63,959,6.57%
в разводе,85,1195,7.11%
женат / замужем,931,12339,7.55%
гражданский брак,388,4150,9.35%
не женат / не замужем,274,2810,9.75%


Customers who are currently or in the past were officially married are more likely to repay loans.

#### Is there a relationship between income level and loan repayment on time?

In [29]:
#create a pivot table and a new column in it
data_income = data.groupby('total_income').agg({'debt': ['sum','count']})
data_income['ratio'] = data_income['debt']['sum']/data_income['debt']['count']*100
display(data_income.sort_values(by = 'ratio').style.format({('ratio', ''):'{:.2f}%'}))

No relationship.

#### How do different purposes of a loan affect its repayment on time?

In [30]:
#create a pivot table and a new column in it
data_purpose = data.pivot_table(index='purpose', values='debt', aggfunc=['sum','count'])
data_purpose['ratio'] = (data_purpose[('sum','debt')] / data_purpose[('count','debt')]) * 100
display(data_purpose.sort_values(by='ratio').style.format({('ratio', ''):'{:.2f}%'}))

,sum,count,ratio
,debt,debt,
purpose,,,
недвижимость,782,10810,7.23%
свадьба,186,2324,8.00%
образование,370,4013,9.22%
автомобиль,403,4306,9.36%


Customers who who take out a loan for transactions with residential or commercial real estate are noticeably more likely to repay loans on time than those who take out a loan for education or a car. The group of customers who take out a loan for a wedding is noticeably smaller than the other three groups, it is not correct to take this group into account.

#### Does customer gender affect loan repayment on time?

In [31]:
#create a pivot table and a new column in it
data_gender = data.groupby('gender').agg({'debt': ['sum','count']})
data_gender['ratio'] = data_gender['debt']['sum']/data_gender['debt']['count']*100
display(data_gender.sort_values(by = 'ratio').style.format({('ratio', ''):'{:.2f}%'}))

Men are less likely to repay their loans on time than women. Among the borrowers the number of women is 2 times more than men.

#### Does the level of education affect the repayment of the loan on time?

In [32]:
#create a pivot table and a new column in it
data_education = data.groupby('education').agg({'debt': ['sum','count']})
data_education['ratio'] = data_education['debt']['sum']/data_education['debt']['count']*100
display(data_education.sort_values(by = 'ratio').style.format({('ratio', ''):'{:.2f}%'}))

People with the degree are more likely to repay the loan on time than people without.